In [1]:
# Show gpu
!nvidia-smi

Wed Dec 10 07:49:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install torch-geometric -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.5 MB/s eta 0:00:00


In [3]:
# config.py
import torch

class config:
    # Cấu hình tham số Model
    HIDDEN_CHANNELS = 64
    LEARNING_RATE = 0.001
    EPOCHS = 50
    DROPOUT = 0.2
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # File paths
    NODE_FILE = '/content/dataset_nodes_info.csv'
    EDGE_FILE = '/content/dataset_topology_edges.csv'
    TICKET_FILE = '/content/dataset_tickets.csv'
    MODEL_PATH = '/content/rca_gnn_model.pth'
    VECTORIZER_PATH = '/content/vectorizer.pkl'

    # Cấu hình Feature
    # Kích thước vector cho text log (Description)
    TEXT_EMBEDDING_DIM = 16

In [4]:
# model.py
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
# import config  <-- Removed import

# --- THÊM CLASS NÀY ---
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # inputs: Logits (chưa qua sigmoid)
        # targets: labels (0 hoặc 1)

        bce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-bce_loss) # pt là xác suất dự đoán đúng
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


class RootCauseGNN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes=1):
        super(RootCauseGNN, self).__init__()

        # Layer 1: GAT Conv
        # heads=4 giúp model học được nhiều mối quan hệ khác nhau
        self.conv1 = GATConv(num_node_features, hidden_channels, heads=4, dropout=config.DROPOUT)

        # Layer 2: GAT Conv
        # Input dim = hidden_channels * heads
        self.conv2 = GATConv(hidden_channels * 4, hidden_channels, heads=2, dropout=config.DROPOUT)

        # Layer 3: Output Layer
        # Trả về 1 giá trị duy nhất (Logit) cho mỗi node để dùng BCEWithLogitsLoss
        self.conv3 = GATConv(hidden_channels * 2, num_classes, heads=1, concat=False, dropout=config.DROPOUT)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Layer 1
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=config.DROPOUT, training=self.training)

        # Layer 2
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=config.DROPOUT, training=self.training)

        # Layer 3
        x = self.conv3(x, edge_index)

        # Trả về Logits (chưa qua Sigmoid)
        return x

In [5]:
# data_processor.py
import pandas as pd
import torch
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from torch_geometric.data import Data
# import config <-- Removed import
import pickle
import os

class TelcoGraphDataset:
    def __init__(self, mode='train'):
        self.node_mapping = {}
        self.reverse_mapping = {}
        self.tfidf = TfidfVectorizer(max_features=config.TEXT_EMBEDDING_DIM, stop_words='english')
        self.type_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

        # 1. Load Data
        self.nodes_df = pd.read_csv(config.NODE_FILE)
        self.edges_df = pd.read_csv(config.EDGE_FILE)

        # Nếu file ticket tồn tại thì load, không thì để trống (cho trường hợp infer sau này)
        if os.path.exists(config.TICKET_FILE):
            self.tickets_df = pd.read_csv(config.TICKET_FILE)
            self.tickets_df['Timestamp'] = pd.to_datetime(self.tickets_df['Timestamp'])
        else:
            self.tickets_df = pd.DataFrame()

        # 2. Xử lý Vectorizer (FIT TRƯỚC khi dùng)
        if mode == 'train':
            print("Fitting Vectorizers on Training Data...")
            # Fit text features
            self.tfidf.fit(self.tickets_df['Description'].fillna(""))
            # Fit node static features
            self.type_encoder.fit(self.nodes_df[['type', 'vendor']])

            # Save vectorizers
            with open(config.VECTORIZER_PATH, 'wb') as f:
                pickle.dump((self.tfidf, self.type_encoder), f)
        else:
            # Mode eval/infer: Load vectorizers đã train
            if os.path.exists(config.VECTORIZER_PATH):
                print("Loading Vectorizers...")
                with open(config.VECTORIZER_PATH, 'rb') as f:
                    self.tfidf, self.type_encoder = pickle.load(f)
            else:
                raise Exception(f"Vectorizer file {config.VECTORIZER_PATH} not found! Run training first.")

        # 3. Sau khi đã có encoder, mới chuẩn bị mapping và static features
        self._prepare_mappings_and_static_data()

    def _prepare_mappings_and_static_data(self):
        # Map Node ID <-> Integer Index
        for idx, row in self.nodes_df.iterrows():
            self.node_mapping[row['id']] = idx
            self.reverse_mapping[idx] = row['id']

        # Xây dựng Edge Index (Cấu trúc đồ thị tĩnh)
        src, dst = [], []
        for _, row in self.edges_df.iterrows():
            if row['Source'] in self.node_mapping and row['Target'] in self.node_mapping:
                u, v = self.node_mapping[row['Source']], self.node_mapping[row['Target']]
                # Đồ thị vô hướng (2 chiều) để tin lan truyền tốt hơn
                src.extend([u, v])
                dst.extend([v, u])
        self.edge_index = torch.tensor([src, dst], dtype=torch.long)

        # Tạo Static Features (Type, Vendor) cho tất cả các node
        # Lúc này self.type_encoder ĐÃ ĐƯỢC FIT rồi, nên gọi transform sẽ không lỗi
        self.static_x = self.type_encoder.transform(self.nodes_df[['type', 'vendor']])

    def create_time_windows(self, window_size_min=10):
        """Chia toàn bộ ticket thành các cửa sổ thời gian"""
        if self.tickets_df.empty:
            return []

        start_time = self.tickets_df['Timestamp'].min()
        end_time = self.tickets_df['Timestamp'].max()

        windows = []
        current = start_time
        while current < end_time:
            next_window = current + pd.Timedelta(minutes=window_size_min)
            # Lọc ticket trong khoảng này
            mask = (self.tickets_df['Timestamp'] >= current) & (self.tickets_df['Timestamp'] < next_window)
            batch_df = self.tickets_df[mask]

            # Chỉ lấy window nào CÓ ticket (để tiết kiệm thời gian train)
            if not batch_df.empty:
                windows.append(batch_df)
            current = next_window

        return windows

    def df_to_graph_data(self, batch_df):
        """Chuyển đổi DataFrame ticket của 1 cửa sổ thời gian thành PyG Data"""
        num_nodes = len(self.nodes_df)
        labels = np.zeros(num_nodes, dtype=float)

        # Dynamic Features (Text Embedding) khởi tạo bằng 0
        dynamic_x = np.zeros((num_nodes, config.TEXT_EMBEDDING_DIM))

        # Map tickets vào node tương ứng
        for _, row in batch_df.iterrows():
            if row['Device_ID'] in self.node_mapping:
                idx = self.node_mapping[row['Device_ID']]

                # Vector hóa Description
                # Dùng transform (không fit lại)
                vec = self.tfidf.transform([row['Description']]).toarray()[0] # type: ignore
                dynamic_x[idx] += vec

                # Gán nhãn Root Cause
                if row.get('Is_Root_Cause', 0) == 1:
                    labels[idx] = 1.0

        # Kết hợp Static Features và Dynamic Features
        # Static (ví dụ 10 chiều) + Dynamic (16 chiều) -> Feature Vector 26 chiều
        final_x = np.hstack([self.static_x, dynamic_x]) # type: ignore

        return Data(
            x=torch.tensor(final_x, dtype=torch.float),
            edge_index=self.edge_index,
            y=torch.tensor(labels, dtype=torch.float)
        )

In [8]:
# main.py
import torch
import torch.nn as nn
from torch_geometric.loader import DataLoader
# import config <-- Removed import
# from data_processor import TelcoGraphDataset <-- Removed import
# from model import RootCauseGNN, FocalLoss <-- Removed import
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import os

def train():
    print(">>> Initializing Dataset...")
    dataset_handler = TelcoGraphDataset(mode='train')

    print(">>> Creating Time Windows...")
    # Cửa sổ 30 phút để gom đủ ngữ cảnh
    all_windows = dataset_handler.create_time_windows(window_size_min=30)

    if len(all_windows) == 0:
        print("ERROR: No data windows created.")
        return

    # Split Train/Test
    split_idx = int(len(all_windows) * 0.8)
    train_windows = all_windows[:split_idx]
    test_windows = all_windows[split_idx:]

    print(f"Total Windows: {len(all_windows)}. Train: {len(train_windows)}, Test: {len(test_windows)}")

    print(">>> Converting to Graphs...")
    train_data_list = [dataset_handler.df_to_graph_data(df) for df in train_windows]
    test_data_list = [dataset_handler.df_to_graph_data(df) for df in test_windows]

    train_loader = DataLoader(train_data_list, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data_list, batch_size=32, shuffle=False)

    # Setup Model
    sample_data = train_data_list[0]
    num_features = sample_data.num_features
    print(f"Model Input Features: {num_features}")

    model = RootCauseGNN(num_features, config.HIDDEN_CHANNELS, num_classes=1).to(config.DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.LEARNING_RATE, weight_decay=1e-4)

    # --- FIX TRỌNG SỐ (POS_WEIGHT) ---
    total_pos = sum([d.y.sum().item() for d in train_data_list])
    total_neg = sum([(d.y == 0).sum().item() for d in train_data_list])

    # CŨ: pos_weight_val = total_neg / (total_pos + 1e-5) -> Ra 14000 (Quá lớn)
    # MỚI: Dùng căn bậc hai để làm mượt (Damping)
    pos_weight_val = np.sqrt(total_neg / (total_pos + 1e-5))

    # Hoặc nếu vẫn cao, gán cứng một con số hợp lý (ví dụ 50.0)
    # pos_weight_val = 50.0

    pos_weight = torch.tensor([pos_weight_val]).to(config.DEVICE)
    print(f"⚖️ Adjusted Pos Weight: {pos_weight.item():.2f}")

    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    print("\n--- START TRAINING ---")
    best_f1 = 0.0

    for epoch in range(config.EPOCHS):
        model.train()
        total_loss = 0

        for batch in train_loader:
            batch = batch.to(config.DEVICE)
            optimizer.zero_grad()
            out = model(batch)
            loss = criterion(out.squeeze(), batch.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)

        if (epoch + 1) % 5 == 0:
            val_metrics = evaluate(model, test_loader)
            print(f"Epoch {epoch+1:03d} | Loss: {avg_loss:.4f} | "
                  f"Val Recall: {val_metrics['recall']:.2f} | "
                  f"Val Prec: {val_metrics['precision']:.2f} | "
                  f"Val F1: {val_metrics['f1']:.2f}")

            # Lưu model nếu F1 cải thiện
            if val_metrics['f1'] > best_f1:
                best_f1 = val_metrics['f1']
                torch.save(model.state_dict(), config.MODEL_PATH)
                print(f"   >>> New Best Model Saved (F1: {best_f1:.2f})")

def evaluate(model, loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in loader:
            batch = batch.to(config.DEVICE)
            out = model(batch)
            probs = torch.sigmoid(out.squeeze())

            # Threshold quan trọng: Tăng lên 0.7 hoặc 0.8 để lọc bớt False Positive
            preds = (probs > 0.7).float()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch.y.cpu().numpy())

    return {
        'recall': recall_score(all_labels, all_preds, zero_division=0),
        'precision': precision_score(all_labels, all_preds, zero_division=0),
        'f1': f1_score(all_labels, all_preds, zero_division=0)
    }

# --- PHẦN INFERENCE SAU KHI TRAIN ---
def run_inference_demo():
    print("\n--- RUNNING DEMO INFERENCE ---")

    # 1. Load Data Handler (Mode test để load vectorizer đã train)
    if not os.path.exists(config.VECTORIZER_PATH):
        print("⚠️ Chưa có file vectorizer, cần train trước!")
        return

    dataset_handler = TelcoGraphDataset(mode='test')

    # 2. Load Model
    # Cần tạo 1 dummy data để lấy num_features khởi tạo model
    dummy_df = pd.DataFrame([{"Device_ID": "DUMMY", "Description": "", "Timestamp": pd.Timestamp.now()}])
    dummy_data = dataset_handler.df_to_graph_data(dummy_df)

    model = RootCauseGNN(dummy_data.num_features, config.HIDDEN_CHANNELS, num_classes=1).to(config.DEVICE)

    if os.path.exists(config.MODEL_PATH):
        model.load_state_dict(torch.load(config.MODEL_PATH, map_location=config.DEVICE))
        print(f"📂 Loaded model from {config.MODEL_PATH}")
    else:
        print("⚠️ Model file not found!")
        return

    model.eval()

    # 3. Định nghĩa Test Cases

    # CASE A: Sự cố thật (Root Cause: AGG-005 bị lỗi OSPF)
    # Kéo theo: ACCESS-0021 (Link Down), ONT-00156 (Lost)
    case_incident = [
        {"Device_ID": "AGG-005", "Description": "OSPF State Change to Down", "Is_Root_Cause": "?"},
        {"Device_ID": "ACCESS-0021", "Description": "Link Down", "Is_Root_Cause": "?"},
        {"Device_ID": "ONT-00156", "Description": "ERROR_Lost 100%", "Is_Root_Cause": "?"},
        {"Device_ID": "CORE-01", "Description": "Interface Up", "Is_Root_Cause": "?"} # Nhiễu
    ]

    # CASE B: Bình thường (Chỉ có bảo trì)
    case_normal = [
        {"Device_ID": "AGG-010", "Description": "Bảo trì định kỳ hệ thống", "Is_Root_Cause": "?"},
        {"Device_ID": "ONT-00888", "Description": "Khách hàng báo chậm", "Is_Root_Cause": "?"}
    ]

    # Hàm phụ trợ để chạy 1 case
    def predict_case(case_name, tickets_list):
        print(f"\nTesting {case_name} ({len(tickets_list)} tickets)...")
        df_batch = pd.DataFrame(tickets_list)
        graph_data = dataset_handler.df_to_graph_data(df_batch)
        graph_data = graph_data.to(config.DEVICE)

        with torch.no_grad():
            out = model(graph_data)
            probs = torch.sigmoid(out.squeeze())

        # In kết quả
        found_root = False
        for idx, prob in enumerate(probs):
            p_val = prob.item()
            if p_val > 0.5: # Threshold hiển thị
                node_id = dataset_handler.reverse_mapping[idx]
                # Lấy description từ input nếu có
                desc = next((t['Description'] for t in tickets_list if t['Device_ID'] == node_id), "N/A (No Ticket)")
                print(f"🔴 ALERT: {node_id} | Prob: {p_val:.4f} | Log: {desc}")
                found_root = True

        if not found_root:
            print("🟢 System Normal (No Root Cause Detected)")

    # Chạy test
    predict_case("CASE A (Sự cố OSPF)", case_incident)
    predict_case("CASE B (Bảo trì)", case_normal)

if __name__ == "__main__":
    if not os.path.exists(config.TICKET_FILE):
        print("Please run generate_data.py first!")
    else:
        # 1. Train
        train()

        # 2. Test Cases
        run_inference_demo()

>>> Initializing Dataset...
Fitting Vectorizers on Training Data...
>>> Creating Time Windows...
Total Windows: 1409. Train: 1127, Test: 282
>>> Converting to Graphs...
Model Input Features: 31
⚖️ Adjusted Pos Weight: 120.86

--- START TRAINING ---
Epoch 005 | Loss: 0.0386 | Val Recall: 0.39 | Val Prec: 0.12 | Val F1: 0.19
   >>> New Best Model Saved (F1: 0.19)
Epoch 010 | Loss: 0.0356 | Val Recall: 0.36 | Val Prec: 0.12 | Val F1: 0.18
Epoch 015 | Loss: 0.0325 | Val Recall: 0.75 | Val Prec: 0.08 | Val F1: 0.15
Epoch 020 | Loss: 0.0299 | Val Recall: 0.34 | Val Prec: 0.19 | Val F1: 0.25
   >>> New Best Model Saved (F1: 0.25)
Epoch 025 | Loss: 0.0286 | Val Recall: 0.75 | Val Prec: 0.19 | Val F1: 0.30
   >>> New Best Model Saved (F1: 0.30)
Epoch 030 | Loss: 0.0266 | Val Recall: 0.82 | Val Prec: 0.07 | Val F1: 0.12
Epoch 035 | Loss: 0.0248 | Val Recall: 0.23 | Val Prec: 0.22 | Val F1: 0.22
Epoch 040 | Loss: 0.0233 | Val Recall: 0.55 | Val Prec: 0.25 | Val F1: 0.35
   >>> New Best Model Save